In [ ]:
import os
import base64
import requests
import json
from PIL import Image

OLLAMA_ENDPOINT = "http://localhost:11434/api/chat"
MODEL_NAME = "llava:7b-v1.6-mistral-q4_0"
IMG_DIR = "img/campus_scenes_jpgs"

def encode_image_to_base64(image_path):
    with open(image_path, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")

def describe_image(image_path):
    image_data = encode_image_to_base64(image_path)

    payload = {
        "model": MODEL_NAME,
        "messages": [
            {
                "role": "user",
                "content": "You are a image analyst. Create a short title of 50 words or less, and a longer descrption of 200 words or less.",
                "images": [image_data]
            }
        ]
    }

    response = requests.post(OLLAMA_ENDPOINT, json=payload)
    response.raise_for_status()

    reply = ""
    for line in response.iter_lines():
        if line:
            data = json.loads(line.decode("utf-8"))
            if "message" in data and "content" in data["message"]:
                reply += data["message"]["content"]
    if not reply:
        raise ValueError("No valid message content received from Ollama.")
    return reply

def test():
    for filename in os.listdir(IMG_DIR):
        if filename.lower().endswith(".jpg"):
            img_path = os.path.join(IMG_DIR, filename)
            print(f"\n🖼️ Analyzing: {filename}")
            try:
                result = describe_image(img_path)
                print(f"🔍 Result:\n{result}")
                with open("results.txt", "a") as out_file:
                    out_file.write(f"Image: {filename}\n{result}\n\n")
                    print('done')
            except Exception as e:
                print(f"❌ Failed to analyze {filename}: {e}")


In [2]:
test()


🖼️ Analyzing: Campusscene005.jpg
🔍 Result:
 Title: "Serene Campus Oasis"

Description: This black and white photograph captures a peaceful moment on a college campus. The focal point is a large concrete planter filled with lush green trees, offering shade to the two individuals who have chosen this spot to sit and relax. On the left side of the image, a person is seated on one of the benches, engrossed in their cell phone, perhaps studying or enjoying some leisure time. To the right, another person is sitting on a bench, facing away from the camera, absorbed in their own world. The presence of other people in the background adds to the sense of tranquility and community that characterizes such educational spaces. Despite being black and white, the image conveys a serene atmosphere, reminiscent of the quiet moments one might find on a campus walkway during a sunny day. 
done

🖼️ Analyzing: BridgeYearBook1965001.jpg
🔍 Result:
 Title: "Serene Schoolyard Under Dusk"

Description: This bla